# Tarea Semana 1 — Dense Layers 
**Equipo:** Alan De Loa, Leonardio Lopez, Isaac Zaragoza, Luis Diaz, Luis Guzman 
**Fecha:** 06-10-2025

**Objetivo:** En esta tarea se busca construir y evaluar un modelo de Deep Learning basado en capas densas con el dataset de cancer de Mama. El objetivo principal es predecir si un tumor es benigno o maligno a partir de 30 caracteristicas numericas obtenidas por imagenes.

## 1) Introducción (5%)

En este notebook modelamos un problema de clasificación binaria usando el dataset Breast Cancer. Este dataset se obtuvo de la libreria de scikit-learn, y muestra diferentes imagenes las cuales se analizan para la detecciones de tumores. Elegimos este dataset por ser tabular, limpio y apropiado para capas densas.

## 2) Exploración, explicación y limpieza de datos (20%)

**Fuente y contexto del dataset:**  
El dataset utilizado proviene de la libreria `scikit-learn`. Este conjunto de datos esta basado en el **Wisconsin Breast Cancer Dataset**, el cual es ampliamente conocido para problemas de clasificacion binaria en el ambito medico.
- La tarea es poder predecir si un tumor es maligno o benigno.
- Se cuetan con 569 muestras de tumores.
- Se tienen 30 caracteristicas numericas obtenidas de imagenes digitales.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, roc_auc_score, confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight

# Reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Utilidad para mostrar separadores bonitos en salidas de consola
def banner(text):
    print("\n" + "="*80)
    print(text)
    print("="*80 + "\n")

In [ ]:
# Carga del dataset
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target  # 0=maligno, 1=benigno

banner("Vista general del DataFrame")
print("Shape (instancias, columnas):", df.shape)   
display(df.head(3))

banner("Información de tipos de datos")
print(df.info())

banner("Verificación de nulos")
print(df.isnull().sum().sum(), "valores nulos en total")

banner("Distribución de la variable objetivo (target: 0=maligno, 1=benigno)")
print(df['target'].value_counts())
print("\nProporción:")
print(df['target'].value_counts(normalize=True).round(3))

Limpieza y Transformación
- Datos faltantes: el dataset no contiene valores nulos (no se requiere imputación).
- Escala: las características tienen escalas muy distintas, lo cual puede afectar la estabilidad del entrenamiento → **Estandarizaremos** con `StandardScaler` (media 0, desviación 1).
- División estratificada: usaremos 80/20 con `stratify=y` para preservar la proporción de clases.
- class_weight: compensará el leve desbalance en entrenamiento.

## 3) Desarrollo del Modelo de Deep Learning (25%)

In [ ]:
# Separar X, y
X = df.drop(columns=['target']).values.astype('float32')  # (569, 30)
y = df['target'].values                                   # (569,)

# Split estratificado 80/20
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=SEED, stratify=y
)

# Escalado estándar (fit solo en train)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw).astype('float32')
X_test  = scaler.transform(X_test_raw).astype('float32')

banner("Shapes tras split y escalado")
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

# Pesos de clase (balanceo)
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weight_dict = {int(c): w for c, w in zip(classes, class_weights)}
banner("Pesos de clase")
print(class_weight_dict)

In [ ]:
# Callbacks compartidos
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=12, restore_best_weights=True
)

reduce_on_plateau = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6, verbose=1
)

In [ ]:
def build_model_A(input_dim: int, seed: int = SEED) -> keras.Model:
    """
    Configuración A (Baseline):
    - Inicialización He + ReLU (estabilidad de gradientes)
    - Regularización L2 + Dropout (evitar sobreajuste)
    - Salida sigmoide (clasificación binaria)
    - LR: Adam(1e-3) + ReduceLROnPlateau para evitar divergencia
    """
    he = keras.initializers.HeNormal(seed=seed)
    l2 = regularizers.l2(1e-4)

    inputs = keras.Input(shape=(input_dim,), name="features")
    x = layers.Dense(64, activation="relu", kernel_initializer=he, kernel_regularizer=l2)(inputs)
    x = layers.Dropout(0.25)(x)
    x = layers.Dense(32, activation="relu", kernel_initializer=he, kernel_regularizer=l2)(x)
    x = layers.Dropout(0.25)(x)
    outputs = layers.Dense(1, activation="sigmoid", kernel_initializer="glorot_uniform")(x)

    model = keras.Model(inputs, outputs, name="DenseBaseline_A")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=[keras.metrics.BinaryAccuracy(name="acc"),
                 keras.metrics.AUC(name="auc")]
    )
    return model

model_A = build_model_A(X_train.shape[1])

history_A = model_A.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=[early_stop, reduce_on_plateau],
    class_weight=class_weight_dict,
    verbose=0
)

banner("Entrenamiento finalizado: Modelo A")
print("Últimas métricas val:", {k: v[-1] for k, v in history_A.history.items() if k.startswith("val_")})

In [ ]:
def build_model_B(input_dim: int, x_train_len: int, seed: int = SEED) -> keras.Model:
    """
    Configuración B (más estable con normalización y enfriamiento coseno):
    - BatchNormalization entre capas densas
    - CosineDecay para LR (enfriamiento coseno)
    - Regularización L2 + Dropout
    """
    he = keras.initializers.HeNormal(seed=seed)
    l2 = regularizers.l2(1e-4)

    # Pasos aproximados para CosineDecay (asumiendo 200 épocas y batch 32, 80% de train para entrenamiento interno)
    steps_per_epoch = int(np.ceil(len(X_train) * 0.8 / 32))
    total_steps = steps_per_epoch * 200
    lr_schedule = keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=1e-3, decay_steps=total_steps
    )

    inputs = keras.Input(shape=(input_dim,))
    x = layers.Dense(96, kernel_initializer=he, kernel_regularizer=l2)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(0.30)(x)

    x = layers.Dense(48, kernel_initializer=he, kernel_regularizer=l2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(0.30)(x)

    outputs = layers.Dense(1, activation="sigmoid", kernel_initializer="glorot_uniform")(x)

    model = keras.Model(inputs, outputs, name="DenseCosine_B")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        loss="binary_crossentropy",
        metrics=[keras.metrics.BinaryAccuracy(name="acc"),
                 keras.metrics.AUC(name="auc")]
    )
    return model

model_B = build_model_B(X_train.shape[1], len(X_train))

history_B = model_B.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=[early_stop],  # CosineDecay ya ajusta LR
    class_weight=class_weight_dict,
    verbose=0
)

banner("Entrenamiento finalizado: Modelo B")
print("Últimas métricas val:", {k: v[-1] for k, v in history_B.history.items() if k.startswith("val_")})

In [ ]:
def best_of(history, metric):
    vals = history.history.get(f"val_{metric}")
    best = float(np.max(vals))
    epoch = int(np.argmax(vals))
    return best, epoch

acc_A, ep_acc_A = best_of(history_A, "acc")
auc_A, ep_auc_A = best_of(history_A, "auc")
acc_B, ep_acc_B = best_of(history_B, "acc")
auc_B, ep_auc_B = best_of(history_B, "auc")

banner("Comparación de validación")
print(f"Config A: best val_acc={acc_A:.4f} @epoch {ep_acc_A}, best val_auc={auc_A:.4f} @epoch {ep_auc_A}")
print(f"Config B: best val_acc={acc_B:.4f} @epoch {ep_acc_B}, best val_auc={auc_B:.4f} @epoch {ep_auc_B}")

# Elegimos el mejor por AUC (más robusto cuando hay leves desbalances)
best_model = model_A if auc_A >= auc_B else model_B
best_name = "A" if best_model is model_A else "B"
print("\nMejor modelo por val_auc:", best_name)

In [ ]:
# Predicción probabilidades y clases
probs = best_model.predict(X_test).ravel()
preds = (probs >= 0.5).astype(int)

# Métricas
test_acc = np.mean(preds == y_test)
test_auc = roc_auc_score(y_test, probs)
cm = confusion_matrix(y_test, preds)
report = classification_report(y_test, preds, target_names=data.target_names)

banner("Evaluación en Test")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test ROC-AUC:  {test_auc:.4f}")
print("\nMatriz de confusión:\n", cm)
print("\nReporte de clasificación:\n", report)


### Curvas de entrenamiento
Graficamos `loss`, `auc` y `accuracy` para comparar configuraciones.


In [ ]:
def plot_history(history, title):
    # Loss
    plt.figure()
    plt.plot(history.history["loss"], label="loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.title(f"{title} - Loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.show()

    # Accuracy
    plt.figure()
    plt.plot(history.history["acc"], label="acc")
    plt.plot(history.history["val_acc"], label="val_acc")
    plt.title(f"{title} - Accuracy")
    plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend(); plt.show()

    # AUC
    plt.figure()
    plt.plot(history.history["auc"], label="auc")
    plt.plot(history.history["val_auc"], label="val_auc")
    plt.title(f"{title} - AUC")
    plt.xlabel("Epoch"); plt.ylabel("AUC"); plt.legend(); plt.show()

plot_history(history_A, "Modelo A")
plot_history(history_B, "Modelo B")

## Justificación de Decisiones de Diseño

### 1️⃣ Función de costo
- **Binary Cross-Entropy** fue utilizada porque el problema es de **clasificación binaria**.  
  Esta función mide la discrepancia entre las probabilidades predichas y las etiquetas reales, penalizando fuertemente predicciones incorrectas con alta confianza.  

### 2️⃣ Inicialización de pesos
- Se usó **He Normal Initialization** para las capas con activación **ReLU**:
  > “La inicialización He ajusta la varianza para evitar gradientes que desaparecen o explotan.”
- La capa de salida usa **Glorot/Xavier** porque la activación `sigmoid` requiere una distribución más balanceada.

### 3️⃣ Función de activación
- **ReLU**: evita saturación (mitiga el problema de gradientes que desaparecen).
- **Sigmoid** en salida: convierte la salida en probabilidad entre 0 y 1.

### 4️⃣ Regularización
- Se aplicaron dos técnicas complementarias:
  - **L2 (weight decay)**: penaliza pesos grandes → evita sobreajuste.
  - **Dropout (25–30%)**: desactiva aleatoriamente neuronas → fuerza robustez.

### 5️⃣ Control de tasa de aprendizaje
- En **Modelo A**, se usó **ReduceLROnPlateau**: reduce la tasa de aprendizaje automáticamente cuando `val_loss` deja de mejorar.
- En **Modelo B**, se usó **CosineDecay**, una estrategia de “enfriamiento coseno” que reduce y aumenta ligeramente la tasa para explorar el espacio de parámetros con mayor estabilidad.

### 6️⃣ Normalización de entradas
- Se usó **StandardScaler** para llevar todas las características a una escala comparable.
- Esto previene que características con magnitudes muy distintas dominen el cálculo de gradientes, lo que podría causar **divergencia numérica**.

### 7️⃣ Métricas de evaluación
- **Accuracy**: mide el desempeño global.
- **AUC (Area Under the Curve)**: más sensible en datasets con leve desbalance; evalúa la capacidad de separación entre clases.  

### 8️⃣ Estrategias contra la divergencia
- **Escalado de datos + inicialización adecuada + control dinámico de la LR** garantizaron una convergencia suave.  
- Las curvas de pérdida muestran descenso estable y sin oscilaciones fuertes, confirmando que las estrategias aplicadas (ReLU, He, LR decay, regularización) **previnieron la divergencia** del entrenamiento.

## 4) Resultados e interpretación (25%)

En este proyecto se entrenaron **dos configuraciones de redes neuronales densas** con distintos hiperparámetros
(inicialización de pesos, regularización y estrategias de tasa de aprendizaje) para clasificar tumores como
**malignos o benignos**.

Tras comparar sus métricas de validación y prueba:

- **Modelo A** (He + ReLU + L2 + Dropout + ReduceLROnPlateau) obtuvo mejor desempeño y mayor estabilidad de entrenamiento (val_AUC = 1.0, test_AUC = 0.9911).  
- **Modelo B** (BatchNorm + CosineDecay) mostró buena convergencia temprana, pero menor val_loss y AUC ligeramente inferiores.

**Interpretación de los resultados:**
- El modelo logra una **separación casi perfecta** entre clases (AUC ≈ 0.99).  
- Solo presenta **4 falsos negativos**, manteniendo alta sensibilidad para casos malignos.  
- Las curvas de pérdida y precisión confirman una **convergencia suave sin divergencia**.

**Justificación del modelo final seleccionado:**
Se eligió el **Modelo A** porque:
1. La combinación de **He initialization + ReLU** mantiene gradientes estables (evita desvanecimiento o explosión).  
2. La regularización **L2 + Dropout** mejora la generalización.  
3. **ReduceLROnPlateau** permite ajustar dinámicamente la tasa de aprendizaje evitando oscilaciones (divergencia).  
4. La métrica **AUC** fue priorizada sobre la precisión global, al ser más robusta en problemas con leve desbalance y donde la clase “maligno” tiene mayor impacto clínico.

## 5) Conclusión (5%)

En este notebook se desarrolló un flujo completo de trabajo en Deep Learning para la **clasificación de cáncer de mama** utilizando el dataset de `scikit-learn`.  
A lo largo del proceso se aplicaron los conceptos teóricos vistos en clase, desde la exploración y limpieza de datos hasta la implementación y evaluación de modelos neuronales.

1. **Exploración y Preparación de Datos:**  
   Se analizaron 569 instancias y 30 características numéricas sin valores nulos.  
   Los datos fueron **escalados con StandardScaler** para garantizar estabilidad numérica y una convergencia adecuada durante el entrenamiento.

2. **Desarrollo del Modelo:**  
   Se entrenaron dos configuraciones de **redes neuronales densas** con diferentes estrategias de inicialización, regularización y control de tasa de aprendizaje.  
   - **Modelo A:** Inicialización *He Normal*, activación *ReLU*, regularización *L2 + Dropout*, optimizador *Adam* con *ReduceLROnPlateau*.  
   - **Modelo B:** Incorporó *Batch Normalization* y *CosineDecay* como ajuste dinámico de la tasa de aprendizaje.

3. **Resultados:**  
   Ambos modelos lograron un desempeño sobresaliente, pero el **Modelo A** destacó con:  
   - **Accuracy (test):** 94.7%  
   - **AUC (test):** 0.9911  
   - Curvas de pérdida y precisión estables, sin señales de divergencia.  

4. **Interpretación:**  
   El modelo logró **alta capacidad de generalización** y excelente separación entre clases.  
   Los pocos falsos negativos indican buena sensibilidad, aspecto crucial en aplicaciones médicas.  


## 6) Orden y buenas prácticas (20%)

- Todo el **contexto** y las **explicaciones** están en **Markdown**.  
- El **código** está **comentado** en las celdas (con `# ...`).  
- El notebook está **limpio** y **ordenado**, con secciones que siguen la **rúbrica**.  
- Se probaron **múltiples configuraciones** y se **justificaron** las decisiones.
